# Get Data & clear previous data

We will clone my github repository that contains the input data and also the pretrained weights

In [12]:
!rm -rf *
!git clone https://github.com/adityagupta1089/Captcha-Generator-using-cGANs.git

Cloning into 'Captcha-Generator-using-cGANs'...
remote: Enumerating objects: 1079, done.
remote: Counting objects: 100% (1079/1079), done.
remote: Compressing objects: 100% (1076/1076), done.
remote: Total 1079 (delta 1), reused 1079 (delta 1), pack-reused 0
Receiving objects: 100% (1079/1079), 20.16 MiB | 21.42 MiB/s, done.
Resolving deltas: 100% (1/1), done.


We will now move the input folder out of clone repo and remove the repo folder

In [0]:
!mv Captcha-Generator-using-cGANs/input/ input/
!rm -rf Captcha-Generator-using-cGANs

# Read Data

We define three functions that will encode and decode our labels and one-hot enconding back and forth. For this we have assigned each character in 5 character label a one-hot vector of size 36 (26 alphabets and 10 numbers).

*   `index(c)` converts `c`(alphabet or number)  to `0-35`
*   `character` converts `v`(number) to `a-z0-9`
*  `one_hot_label(label)` converts `label` to vector of size $36*5=180$
*  `get_label(one_hot)` converts vector of size `180` to string of size `5`



In [0]:
ALPHABETS = 26
NUMBERS = 10
CHARACTERS = 5
TOTAL = ALPHABETS + NUMBERS

def index(c):
    v = ord(c)
    if 'a' <= c and c <= 'z':
        return v - ord('a')
    elif '0' <= c and c <= '9':
        return ALPHABETS + v - ord('0')
    else:
        return TOTAL + 1
    
def character(v):
    if 0 <= v and v <= ALPHABETS - 1:
        return chr(v + ord('a'))
    elif ALPHABETS <= v and v <= TOTAL - 1:
        return chr(v - ALPHABETS + ord('0'))
    else:
        return '?'

def one_hot_label(label):
    result = []
    for c in label:
        char_result = [0] * TOTAL
        char_result[index(c)] = 1
        result.append(char_result)
    return result

def get_label(one_hot):
    result = ""
    for i in range(0, 180, 36):
        v = np.argmax(one_hot[i:i+36])
        result = result + character(v)
    return result

Now we read all data and create our two input arrays `X` and `Y`

In [31]:
import cv2
from PIL import Image as image
from os import listdir
import numpy as np

def loadData(path):
    images = listdir(path)
    X = []
    Y = []
    for y in images:
        x = cv2.imread(path + y)
        X.append(x)
        Y.append(one_hot_label(y.split('.')[0]))
        
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

path = "input/"

X, Y = loadData(path)

img_shape = X.shape[1:]
print(X.shape)
print(Y.shape)


(1070, 50, 200, 3)
(1070, 5, 36)


#Train Test Split

We split the data into test and train in ratio of 20:80

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    test_size=0.2, 
                                                    random_state=0)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(856, 50, 200, 3)
(214, 50, 200, 3)
(856, 5, 36)
(214, 5, 36)


We also split `Y` into `5` different arrays for each output

In [0]:
def split_fun(arr):
    N = arr.shape[0]
    result = []
    for Yi in np.split(arr, CHARACTERS, axis=1):
        result.append(Yi.reshape(N, TOTAL))
    return result

def combine_fun(arr):
    return np.hstack(tuple(arr))

In [39]:
Y_train = split_fun(Y_train)
Y_test = split_fun(Y_test)

print([y.shape for y in Y_train])
print([y.shape for y in Y_test])

[(856, 36), (856, 36), (856, 36), (856, 36), (856, 36)]
[(214, 36), (214, 36), (214, 36), (214, 36), (214, 36)]


# Define Models

## Solver

### Model

Now we define a model similar to VGGnet

In [44]:
from keras.models import Sequential, Model
from keras.layers import (
     Dense, 
     Conv2D,
     Input,
     BatchNormalization, 
     MaxPooling2D, 
     Flatten, 
     Activation, 
     Dropout)
from keras import backend as K
import tensorflow as tf


def my_solver():
    model = Sequential(name="ConvPart")
    
    input_shape = X.shape[1:]
    
    model.add(Conv2D(64, (5, 5), input_shape=input_shape, 
                     activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
        
    model_input = Input(shape=input_shape, name="Input")
    
    outputs = []
    
    for i in range(0, CHARACTERS):
        outputs.append(Dense(TOTAL, activation='softmax', name="D" + str(i))(model(model_input)))
        
    complete_model = Model(model_input, outputs)
    
    losses = ['categorical_crossentropy'] * CHARACTERS
    
    complete_model.compile(loss=losses, metrics=['categorical_accuracy'], optimizer='adam')
    
    return complete_model

solver = my_solver()
solver.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 50, 200, 3)   0                                            
__________________________________________________________________________________________________
ConvPart (Sequential)           (None, 1024)         3087552     Input[0][0]                      
                                                                 Input[0][0]                      
                                                                 Input[0][0]                      
                                                                 Input[0][0]                      
                                                                 Input[0][0]                      
__________________________________________________________________________________________________
D0 (Dense)

### Training

In [69]:
h = solver.fit(X_train, 
           Y_train, 
           batch_size=8, 
           epochs=150, 
           verbose=1, 
           validation_data=(X_test, Y_test), 
           shuffle=True)

Train on 856 samples, validate on 214 samples
Epoch 1/150
856/856 [==============================] - 13s 15ms/step - loss: 4.9930 - D0_loss: 0.8589 - D1_loss: 0.8954 - D2_loss: 1.1580 - D3_loss: 1.0249 - D4_loss: 1.0557 - D0_categorical_accuracy: 0.7313 - D1_categorical_accuracy: 0.7220 - D2_categorical_accuracy: 0.6507 - D3_categorical_accuracy: 0.6682 - D4_categorical_accuracy: 0.6659 - val_loss: 60.4121 - val_D0_loss: 10.9920 - val_D1_loss: 12.4194 - val_D2_loss: 15.1853 - val_D3_loss: 12.2178 - val_D4_loss: 9.5976 - val_D0_categorical_accuracy: 0.2196 - val_D1_categorical_accuracy: 0.1449 - val_D2_categorical_accuracy: 0.0327 - val_D3_categorical_accuracy: 0.1542 - val_D4_categorical_accuracy: 0.2944
Epoch 2/150
856/856 [==============================] - 12s 14ms/step - loss: 3.2814 - D0_loss: 0.4452 - D1_loss: 0.6378 - D2_loss: 0.7258 - D3_loss: 0.8508 - D4_loss: 0.6219 - D0_categorical_accuracy: 0.8563 - D1_categorical_accuracy: 0.8014 - D2_categorical_accuracy: 0.7815 - D3_categ

We can save weights if we want

In [0]:
solver.save_weights('solver.hdf5')

Or load weights if we want

In [0]:
solver.load_weights('solver.hdf5')

Let us see what the incorrectly labeled captchas look like

In [70]:
solver.evaluate(X_test, 
           Y_test, 
           batch_size=8, 
           verbose=1)

Y_pred = solver.predict(X_test)

def contendors(y):
    top = y.argsort()[-3:][::-1]
    return ', '.join([character(x) + "(" + ("%2.1f%%" % (100.0 * y[x])) + ")" for x in top])

i = 0
for y_test, y_pred in zip(combine_fun(Y_test), combine_fun(Y_pred)):
    label_test = get_label(y_test)
    label_pred = get_label(y_pred)
    if label_test != label_pred:
        i = i + 1        
        print(i, "Wrong: ", label_test, label_pred)
        for j in range(len(label_test)):
            if label_test[j] != label_pred[j]:
                print('\tposition', str(j+1) + ',', contendors(np.split(y_pred, CHARACTERS)[j]))    

214/214 [==============================] - 1s 4ms/step
1 Wrong:  wyc25 w4yc5
	position 2, 4(46.7%), y(18.4%), f(17.4%)
	position 3, y(99.7%), c(0.3%), e(0.0%)
	position 4, c(73.1%), 2(26.9%), w(0.0%)
2 Wrong:  c6f8g c6f8y
	position 5, y(92.3%), p(4.9%), b(1.1%)
3 Wrong:  bf52c bf5cc
	position 4, c(95.8%), 2(4.2%), 3(0.0%)
4 Wrong:  25257 25557
	position 3, 5(99.9%), 2(0.1%), 3(0.0%)
5 Wrong:  cx3wg ex3wg
	position 1, e(99.6%), x(0.3%), w(0.0%)
6 Wrong:  mmg38 mmgp8
	position 4, p(79.1%), g(10.2%), 3(6.2%)
7 Wrong:  ewcf5 ewc35
	position 4, 3(91.3%), c(5.2%), g(1.4%)
8 Wrong:  d22n7 d22m7
	position 4, m(86.0%), n(10.0%), y(1.4%)
9 Wrong:  mwdf6 mwwd6
	position 3, w(100.0%), x(0.0%), 4(0.0%)
	position 4, d(77.4%), w(22.4%), b(0.2%)
10 Wrong:  mxyxw mxyww
	position 4, w(80.7%), x(10.7%), c(6.9%)
11 Wrong:  m4g8g m44w8
	position 3, 4(91.3%), g(7.7%), x(0.7%)
	position 4, w(81.3%), g(13.9%), 8(2.3%)
	position 5, 8(99.6%), g(0.4%), n(0.0%)
12 Wrong:  w52fn wp5fn
	position 2, p(29.1%), f(25.6